In [1]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

import numpy as np
import pandas as pd

import os

In [2]:
def read_data_books():
    data_dir = "M:\\git\\ML2_2021\\data"
    names = ['user_id', 'isbn', 'rating']
    data = pd.read_csv(os.path.join(data_dir, 'book_ratings.csv'), ',', names=names,
                       engine='python', skiprows=1)

    data['isbn'] = data.isbn.astype('category')
    data['book_id'] = data['isbn'].cat.codes

    data['user_id'], _ = pd.factorize(data['user_id'])
    data['book_id'], _ = pd.factorize(data['book_id'])

    data.user_id = data.user_id
    data.book_id = data.book_id
    
    data.drop('isbn',axis=1,inplace=True)
    data = data[['user_id','book_id','rating']]
    
    data['rating'] = data['rating'].astype(float)

    num_users = data.user_id.unique().shape[0]
    num_items = data.book_id.unique().shape[0]
    return data, num_users, num_items

In [3]:
data,nu,ni=read_data_books()

C:\Users\ARICK_~1\AppData\Local\Temp/ipykernel_133676/2596294887.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  data,nu,ni=read_data_books()


In [4]:
# replace this with your data loader

def tensor_data(data, n_user, n_item):  #@save
    users = torch.tensor(data['user_id'],dtype=torch.long)
    books = torch.tensor(data['book_id'],dtype=torch.long)
    rting = torch.tensor(data['rating'],dtype=torch.float32)                              

    X = torch.cat((users.reshape((-1,1)), books.reshape((-1,1))), 1)
    y = rting.reshape((-1,1))
    return X, y

features, labels = tensor_data(data,nu,ni)

In [5]:
_,f_train,f_test=torch.tensor_split(features,(0,round(len(features)*0.8)))
_,l_train,l_test=torch.tensor_split(labels,(0,round(len(features)*0.8)))

In [6]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # The examples are read at random, in no particular order
    # random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i:min(i +
                                                   batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

In [7]:
# replace this with the recsys model
def alfa_only(X, alfa):  #@save
    """The linear regression model."""
    return alfa.repeat(len(X)).reshape(X[:,0].shape)

# replace this with the recsys model
def alfa_user(X, alfa, beta_u):  #@save
    """The linear regression model."""
    return alfa + beta_u[X[:,0]]

# replace this with the recsys model
def alfa_item(X, alfa, beta_i):  #@save
    """The linear regression model."""
    return alfa + beta_i[X[:,1]]

# replace this with the recsys model
def whole_nine_yards(X, alfa, beta_u, beta_i):  #@save
    """The linear regression model."""
    return alfa + beta_u[X[:,0]] + beta_i[X[:,1]]

In [8]:
def squared_loss(y_hat, y):  #@save
    """Squared loss."""
    return (y_hat - y.reshape(y_hat.shape))**2 / 2

In [9]:
def sgd(params, lr, batch_size):  #@save
    """Minibatch stochastic gradient descent."""
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

In [10]:
# driver code
alfa = torch.zeros(1, requires_grad=True)
#beta_u = torch.normal(0, 0.01, size=(nu, 1), requires_grad=True)
#beta_i = torch.normal(0, 0.01, size=(ni, 1), requires_grad=True)

batch_size = 262144

lr = 2.5
num_epochs = 2000
net = alfa_only
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, f_train, l_train):
        l = loss(net(X, alfa), y)  # Minibatch loss in `X` and `y`
        # Compute gradient on `l` with respect to [`w`, `b`]
        l.sum().backward()
        sgd([alfa], lr, batch_size)  # Update parameters using their gradient
    with torch.no_grad():
        train_loss = loss(net(f_train, alfa), l_train)
        test_loss = loss(net(f_test, alfa), l_test)
        print(f'epoch {epoch + 1}, RMSE train: {float(train_loss.mean())**(1/2):f}, RMSE test: {float(test_loss.mean())**(1/2):f}', end = "\r")

In [11]:
# driver code
alfa = torch.zeros(1, requires_grad=True)
beta_u = torch.normal(0, 0.01, size=(nu, 1), requires_grad=True)
# beta_i = torch.normal(0, 0.01, size=(ni, 1), requires_grad=True)

batch_size = 262144

lr = 2.5
num_epochs = 2000
net = alfa_user
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, f_train, l_train):
        l = loss(net(X, alfa, beta_u), y)  # Minibatch loss in `X` and `y`
        # Compute gradient on `l` with respect to [`w`, `b`]
        l.sum().backward()
        sgd([alfa, beta_u], lr, batch_size)  # Update parameters using their gradient
    with torch.no_grad():
        train_loss = loss(net(f_train, alfa, beta_u), l_train)
        test_loss = loss(net(f_test, alfa, beta_u), l_test)
        print(f'epoch {epoch + 1}, RMSE train: {float(train_loss.mean())**(1/2):f}, RMSE test: {float(test_loss.mean())**(1/2):f}', end = "\r")

In [12]:
# driver code
alfa = torch.zeros(1, requires_grad=True)
# beta_u = torch.normal(0, 0.01, size=(nu, 1), requires_grad=True)
beta_i = torch.normal(0, 0.01, size=(ni, 1), requires_grad=True)

batch_size = 262144

lr = 2.5
num_epochs = 2000
net = alfa_item
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, f_train, l_train):
        l = loss(net(X, alfa, beta_i), y)  # Minibatch loss in `X` and `y`
        # Compute gradient on `l` with respect to [`w`, `b`]
        l.sum().backward()
        sgd([alfa, beta_i], lr, batch_size)  # Update parameters using their gradient
    with torch.no_grad():
        train_loss = loss(net(f_train, alfa, beta_i), l_train)
        test_loss = loss(net(f_test, alfa, beta_i), l_test)
        print(f'epoch {epoch + 1}, RMSE train: {float(train_loss.mean())**(1/2):f}, RMSE test: {float(test_loss.mean())**(1/2):f}', end = "\r")

In [13]:
# driver code
alfa = torch.zeros(1, requires_grad=True)
beta_u = torch.normal(0, 0.01, size=(nu, 1), requires_grad=True)
beta_i = torch.normal(0, 0.01, size=(ni, 1), requires_grad=True)

batch_size = 262144

lr = 2.5
num_epochs = 2000
net = whole_nine_yards
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, f_train, l_train):
        l = loss(net(X, alfa, beta_u, beta_i), y)  # Minibatch loss in `X` and `y`
        # Compute gradient on `l` with respect to [`w`, `b`]
        l.sum().backward()
        sgd([alfa, beta_u, beta_i], lr, batch_size)  # Update parameters using their gradient
    with torch.no_grad():
        train_loss = loss(net(f_train, alfa, beta_u, beta_i), l_train)
        test_loss = loss(net(f_test, alfa, beta_u, beta_i), l_test)
        print(f'epoch {epoch + 1}, RMSE train: {float(train_loss.mean())**(1/2):f}, RMSE test: {float(test_loss.mean())**(1/2):f}', end = "\r")